# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [1]:
%%bash 
cd ../ 
python setup.py install 

running install
running bdist_egg
running egg_info
writing facenet_pytorch.egg-info/PKG-INFO
writing dependency_links to facenet_pytorch.egg-info/dependency_links.txt
writing requirements to facenet_pytorch.egg-info/requires.txt
writing top-level names to facenet_pytorch.egg-info/top_level.txt
reading manifest file 'facenet_pytorch.egg-info/SOURCES.txt'
writing manifest file 'facenet_pytorch.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/facenet_pytorch
copying build/lib/facenet_pytorch/__init__.py -> build/bdist.linux-x86_64/egg/facenet_pytorch
copying build/lib/facenet_pytorch/train.py -> build/bdist.linux-x86_64/egg/facenet_pytorch
creating build/bdist.linux-x86_64/egg/facenet_pytorch/data
copying build/lib/facenet_pytorch/data/rnet.pt -> build/bdist.linux-x86_64/egg/facenet_pytorch/data
copying build/lib/facenet_pytorch/data/onet.pt -> buil

package init file './models/__init__.py' not found (or not a regular file)
package init file './models/utils/__init__.py' not found (or not a regular file)
package init file './data/__init__.py' not found (or not a regular file)
zip_safe flag not set; analyzing archive contents...
facenet_pytorch.models.__pycache__.mtcnn.cpython-36: module references __file__


In [2]:
!pip install tensorboard==1.14

     |████████████████████████████████| 3.1 MB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 64.2 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.9 MB/s  eta 0:00:01


In [8]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/hom

#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [ ]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
!tar -xvzf lfw.tgz   

In [6]:
import os
# data_dir = '../data/test_images'
data_dir = './lfw'

batch_size = 32
epochs = 50
workers = 0 if os.name == 'nt' else 8

#### Determine if an nvidia GPU is available

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [10]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [11]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped/'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/facenet_pytorch-2.5.2-py3.6.egg/facenet_pytorch/models/utils/detect_face.py:183: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  batch_boxes, batch_points = np.array(batch_boxes), np.array(batch_points)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/facenet_pytorch-2.5.2-py3.6.egg/facenet_pytorch/models/mtcnn.py:339: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  boxes = np.array(boxes)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/facenet_

Batch 414 of 414

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [12]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

#### Define optimizer, scheduler, dataset, and dataloader

In [13]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [14]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [15]:
print(epochs)
epochs = 8 

50


In [17]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |    83/83   | loss:    7.0186 | fps:  278.6153 | acc:    0.3537   

Epoch 1/50
----------
Train |   331/331  | loss:    0.2863 | fps:   77.4913 | acc:    0.9913   
Valid |    83/83   | loss:    7.0372 | fps:  249.4612 | acc:    0.3562   

Epoch 2/50
----------
Train |   331/331  | loss:    0.2734 | fps:   79.2540 | acc:    0.9924   
Valid |    83/83   | loss:    7.0021 | fps:  284.1617 | acc:    0.3529   

Epoch 3/50
----------
Train |   331/331  | loss:    0.2664 | fps:   79.4581 | acc:    0.9933   
Valid |    83/83   | loss:    7.0096 | fps:  284.6588 | acc:    0.3540   

Epoch 4/50
----------
Train |   331/331  | loss:    0.2579 | fps:   79.4137 | acc:    0.9937   
Valid |    83/83   | loss:    7.0385 | fps:  282.7520 | acc:    0.3536   

Epoch 5/50
----------
Train |   331/331  | loss:    0.2464 | fps:   79.4144 | acc:    0.9942   
Valid |    83/83   | loss:    7.0217 | fps:  281.1966 | acc:    0.3570   

Epoch 6/50
----------
Train |   331/331  | loss: 

KeyboardInterrupt: 

In [18]:
torch.save(resnet, "fine_tuned_model")